In [336]:
import pandas as pd
import numpy as np
import sklearn

In [404]:
# Load the data
df = pd.read_csv('../../../mapped_ids_data.csv')
df.columns = df.columns.str.strip()

In [405]:
df.head()

,Unnamed: 0,PUID,EUID,Age (years),Sex,Height (cm),Weight (kg),BSA (m^2),BMI (kg/m^2),Systolic BP (mmHg),...,Heart failure admission event,Heart Failure Event Date,Death event,Death date,Pateint last seen date,End Point,EndPointFinalized,id,folder,Dataset_type
0,129,a8753562d,bc48a5928,69,Male,182,74,1.96,22.3,122,...,1,10/30/2020,0,NaN,1/30/2022,1,1,285,a8753562d_bc48a5928,Train
1,163,a0c358f13,b3a9d0750,87,Female,171,81,1.94,27.7,157,...,1,7/8/2020,0,NaN,2/18/2022,1,1,284,a0c358f13_b3a9d0750,Train
2,110,a87dac184,bedc4b695,93,Male,170,93,2.04,32.2,149,...,0,NaN,0,NaN,4/6/2022,0,0,283,a87dac184_bedc4b695,Train
3,158,a2d7976c2,bdf182d7f,77,Male,180,107,2.30,33.0,140,...,0,NaN,0,NaN,3/28/2022,0,0,282,a2d7976c2_bdf182d7f,Train
4,194,a09ee302f,bdb37df3d,88,Male,178,109,2.30,34.4,136,...,0,NaN,0,NaN,2/11/2022,0,0,281,a09ee302f_bdb37df3d,Train


In [406]:
# Check for missing values
"""
There are 65 missing Post-procedural aortic insufficiency <- Drop
There are 30 missing Mean Pulmonary Artery pressure (mmHg) <- Mean
There are 30 missing Peak Pulmonary Artery pressure (mmHg) <- Mean
There are 11 missing Baseline TAPSE (mm) <- Mean
There are 3 missing Baseline Degree of Mitral regurgitation <- Mean
There are 3 missing Baseline Degree of tricuspid regurgitation <- Mean
There are 154 missing Heart Failure Event Date <- Drop
There are 174 missing Death date <- Drop
There are 22 missing Pateint last seen date <- Drop
"""
df.isnull().sum()

Unnamed: 0           0
PUID                 0
EUID                 0
Age (years)          0
Sex                  0
                    ..
End Point            0
EndPointFinalized    0
id                   0
folder               0
Dataset_type         0
Length: 115, dtype: int64

In [408]:
# Seperate x and y
y = df.loc[:, ['EndPointFinalized']]
x = df.drop(['EndPointFinalized'], axis=1)

In [409]:
y

,EndPointFinalized
0,1
1,1
2,0
3,0
4,0
...,...
191,0
192,1
193,0
194,0



There are 65 missing Post-procedural aortic insufficiency <- Drop
There are 30 missing Mean Pulmonary Artery pressure (mmHg) <- Mean
There are 30 missing Peak Pulmonary Artery pressure (mmHg) <- Mean
There are 11 missing Baseline TAPSE (mm) <- Mean
There are 3 missing Baseline Degree of Mitral regurgitation <- Mean
There are 3 missing Baseline Degree of tricuspid regurgitation <- Mean
There are 154 missing Heart Failure Event Date <- Drop
There are 174 missing Death date <- Drop
There are 22 missing Pateint last seen date <- Drop

In [410]:
fill_by_mean = [
    'Mean Pulmonary Artery pressure (mmHg)',
    'Peak Pulmonary Artery pressure (mmHg)',
    'Baseline TAPSE (mm)'
]
fill_by_mode = [
    'Baseline Degree of Mitral regurgitation',
    'Baseline Degree of tricuspid regurgitation'
]
for column in fill_by_mean:
    x[column] = x[column].fillna(x[column].mean())

for column in fill_by_mode:
    x[column] = x[column].fillna(x[column].mode()[0])

In [411]:
for c in x.columns:
    print(c)

Unnamed: 0
PUID
EUID
Age (years)
Sex
Height (cm)
Weight (kg)
BSA (m^2)
BMI (kg/m^2)
Systolic BP (mmHg)
Diastolic BP (mmHg)
Heart Rate (bpm)
Diabetes
Hypertension
Hyperlipidemia
Smoking
Prior Myocardial Infarction
Prior PCI
Prior CABG
Prior Stroke or TIA
Peripheral Vascular Disease
Pulmonary Hypertentsion
Chronic obstructive Lung Disease
Home O2 Dependent
Renal insufficiency
Dialysis
Atrial fibrillation or flutter
NYHA functional class
CCS class
Syncope
Society of Thoracic Surgoens STS Score (%)
Creatinine
eGFR
Obstructive coronary artery disease ≥ 2 vessels
Peak Pulmonary Artery pressure (mmHg)
Mean Pulmonary Artery pressure (mmHg)
Access Route
Valve Type
Post-procedural aortic insufficiency
Post-procedural aortic peak gradient (mmHg)
Post-procedural aortic mean gradient (mmHg)
Baseline left ventricle ejection fraction (%)
Baseline LV mass index (g/m2)
Baseline LA volume index (ml/m2)
Baseline Aortic valve peak gradient (mmHg)
Baseline Aortic valve mean gradient (mmHg)
Baseline Aortic 

In [412]:
[x.strip() for x in """Baseline LV mass index (g/m2)
    Baseline LA volume index (ml/m2) 
    Baseline Aortic valve peak gradient (mmHg)
    Baseline Aortic valve mean gradient (mmHg)
    Baseline Aortic valve area index (cm2/m2)
    Baseline TAPSE (mm)
    Baseline Degree of Mitral regurgitation
    Baseline Degree of tricuspid regurgitation 
    Global peak endocardial minimum principal strain (%)
    Endocardial minimum principal strain Segement 1 (%)
    Endocardial minimum principal strain Segement 2 (%)
    Endocardial minimum principal strain Segement 3 (%)
    Endocardial minimum principal strain Segement 4 (%)
    Endocardial minimum principal strain Segement 5 (%)
    Endocardial minimum principal strain Segement 6 (%)
    Endocardial minimum principal strain Segement 7 (%)
    Endocardial minimum principal strain Segement 8 (%)
    Endocardial minimum principal strain Segement 9 (%)
    Endocardial minimum principal strain Segement 10 (%)
    Endocardial minimum principal strain Segement 11 (%)
    Endocardial minimum principal strain Segement 12 (%)
    Endocardial minimum principal strain Segement 13 (%)
    Endocardial minimum principal strain Segement 14 (%)
    Endocardial minimum principal strain Segement 15 (%)
    Endocardial minimum principal strain Segement 16 (%)
    Endocardial minimum principal strain Segement 17 (%)
    Global peak epicardial minimum principal strain (%)
    Epicardial minimum principal strain Segement 1 (%)
    Epicardial minimum principal strain Segement 2 (%)
    Epicardial minimum principal strain Segement 3 (%)
    Epicardial minimum principal strain Segement 4 (%)
    Epicardial minimum principal strain Segement 5 (%)
    Epicardial minimum principal strain Segement 6 (%)
    Epicardial minimum principal strain Segement 7 (%)
    Epicardial minimum principal strain Segement 8 (%)
    Epicardial minimum principal strain Segement 9 (%)
    Epicardial minimum principal strain Segement 10 (%)
    Epicardial minimum principal strain Segement 11 (%)
    Epicardial minimum principal strain Segement 12 (%)
    Epicardial minimum principal strain Segement 13 (%)
    Epicardial minimum principal strain Segement 14 (%)
    Epicardial minimum principal strain Segement 15 (%)
    Epicardial minimum principal strain Segement 16 (%)
    Epicardial minimum principal strain Segement 17 (%)
    Global peak transmural minimum principal strain (%)
    Transmural minimum principal strain Segment 1 (%)
    Transmural minimum principal strain Segment 2 (%)
    Transmural minimum principal strain Segment 3 (%)
    Transmural minimum principal strain Segment 4 (%)
    Transmural minimum principal strain Segment 5 (%)
    Transmural minimum principal strain Segment 6 (%)
    Transmural minimum principal strain Segment 7 (%)
    Transmural minimum principal strain Segment 8 (%)
    Transmural minimum principal strain Segment 9 (%)
    Transmural minimum principal strain Segment 10 (%)
    Transmural minimum principal strain Segment 11 (%)
    Transmural minimum principal strain Segment 12 (%)
    Transmural minimum principal strain Segment 13 (%)
    Transmural minimum principal strain Segment 14 (%)
    Transmural minimum principal strain Segment 15 (%)
    Transmural minimum principal strain Segment 16 (%)
    Transmural minimum principal strain Segment 17 (%)""".split("\n")]

['Baseline LV mass index (g/m2)',
 'Baseline LA volume index (ml/m2)',
 'Baseline Aortic valve peak gradient (mmHg)',
 'Baseline Aortic valve mean gradient (mmHg)',
 'Baseline Aortic valve area index (cm2/m2)',
 'Baseline TAPSE (mm)',
 'Baseline Degree of Mitral regurgitation',
 'Baseline Degree of tricuspid regurgitation',
 'Global peak endocardial minimum principal strain (%)',
 'Endocardial minimum principal strain Segement 1 (%)',
 'Endocardial minimum principal strain Segement 2 (%)',
 'Endocardial minimum principal strain Segement 3 (%)',
 'Endocardial minimum principal strain Segement 4 (%)',
 'Endocardial minimum principal strain Segement 5 (%)',
 'Endocardial minimum principal strain Segement 6 (%)',
 'Endocardial minimum principal strain Segement 7 (%)',
 'Endocardial minimum principal strain Segement 8 (%)',
 'Endocardial minimum principal strain Segement 9 (%)',
 'Endocardial minimum principal strain Segement 10 (%)',
 'Endocardial minimum principal strain Segement 11 (%)',

In [413]:
# Remove unwanted metadata
x = x.drop([
    "Weight (kg)",
    "Height (cm)",
    "Society of Thoracic Surgoens STS Score (%)",
    "Peak Pulmonary Artery pressure (mmHg)",
    "Mean Pulmonary Artery pressure (mmHg)",
    "Post-procedural aortic insufficiency", 
    "Post-procedural aortic peak gradient (mmHg)",
    "Post-procedural aortic mean gradient (mmHg)",
    "Access Route",
    "Valve Type",
    "Systolic BP (mmHg)",
    "Diastolic BP (mmHg)",
    "Heart Rate (bpm)",
    'PUID', 
    "EUID", 
    'Access Route', 
    'Heart Failure Event Date', 
    'Heart failure admission event', 
    'Death event', 
    'Death date', 
    'Pateint last seen date',
    'End Point',
    'folder',
    'Dataset_type',
    'Unnamed: 0',
    'TAVR Surgery Date',
    'Post-procedural aortic insufficiency',
    'id',
    'BMI (kg/m^2)',
    'eGFR',
    'Creatinine',
    'CCS class',
    'NYHA functional class',
] + [x.strip() for x in """Baseline LV mass index (g/m2)
    Baseline LA volume index (ml/m2) 
    Baseline Aortic valve peak gradient (mmHg)
    Baseline Aortic valve mean gradient (mmHg)
    Baseline Aortic valve area index (cm2/m2)
    Baseline TAPSE (mm)
    Baseline Degree of Mitral regurgitation
    Baseline Degree of tricuspid regurgitation 
    Global peak endocardial minimum principal strain (%)
    Endocardial minimum principal strain Segement 1 (%)
    Endocardial minimum principal strain Segement 2 (%)
    Endocardial minimum principal strain Segement 3 (%)
    Endocardial minimum principal strain Segement 4 (%)
    Endocardial minimum principal strain Segement 5 (%)
    Endocardial minimum principal strain Segement 6 (%)
    Endocardial minimum principal strain Segement 7 (%)
    Endocardial minimum principal strain Segement 8 (%)
    Endocardial minimum principal strain Segement 9 (%)
    Endocardial minimum principal strain Segement 10 (%)
    Endocardial minimum principal strain Segement 11 (%)
    Endocardial minimum principal strain Segement 12 (%)
    Endocardial minimum principal strain Segement 13 (%)
    Endocardial minimum principal strain Segement 14 (%)
    Endocardial minimum principal strain Segement 15 (%)
    Endocardial minimum principal strain Segement 16 (%)
    Endocardial minimum principal strain Segement 17 (%)
    Global peak epicardial minimum principal strain (%)
    Epicardial minimum principal strain Segement 1 (%)
    Epicardial minimum principal strain Segement 2 (%)
    Epicardial minimum principal strain Segement 3 (%)
    Epicardial minimum principal strain Segement 4 (%)
    Epicardial minimum principal strain Segement 5 (%)
    Epicardial minimum principal strain Segement 6 (%)
    Epicardial minimum principal strain Segement 7 (%)
    Epicardial minimum principal strain Segement 8 (%)
    Epicardial minimum principal strain Segement 9 (%)
    Epicardial minimum principal strain Segement 10 (%)
    Epicardial minimum principal strain Segement 11 (%)
    Epicardial minimum principal strain Segement 12 (%)
    Epicardial minimum principal strain Segement 13 (%)
    Epicardial minimum principal strain Segement 14 (%)
    Epicardial minimum principal strain Segement 15 (%)
    Epicardial minimum principal strain Segement 16 (%)
    Epicardial minimum principal strain Segement 17 (%)
    Global peak transmural minimum principal strain (%)
    Transmural minimum principal strain Segment 1 (%)
    Transmural minimum principal strain Segment 2 (%)
    Transmural minimum principal strain Segment 3 (%)
    Transmural minimum principal strain Segment 4 (%)
    Transmural minimum principal strain Segment 5 (%)
    Transmural minimum principal strain Segment 6 (%)
    Transmural minimum principal strain Segment 7 (%)
    Transmural minimum principal strain Segment 8 (%)
    Transmural minimum principal strain Segment 9 (%)
    Transmural minimum principal strain Segment 10 (%)
    Transmural minimum principal strain Segment 11 (%)
    Transmural minimum principal strain Segment 12 (%)
    Transmural minimum principal strain Segment 13 (%)
    Transmural minimum principal strain Segment 14 (%)
    Transmural minimum principal strain Segment 15 (%)
    Transmural minimum principal strain Segment 16 (%)
    Transmural minimum principal strain Segment 17 (%)""".split("\n")], axis=1)

In [414]:
x

,Age (years),Sex,BSA (m^2),Diabetes,Hypertension,Hyperlipidemia,Smoking,Prior Myocardial Infarction,Prior PCI,Prior CABG,...,Pulmonary Hypertentsion,Chronic obstructive Lung Disease,Home O2 Dependent,Renal insufficiency,Dialysis,Atrial fibrillation or flutter,Syncope,Obstructive coronary artery disease ≥ 2 vessels,Baseline left ventricle ejection fraction (%),EndPointFinalized
0,69,Male,1.96,Y,Y,Y,Current,1,1,1,...,Y,N,N,N,N,Y,N,1,40.0,1
1,87,Female,1.94,Y,Y,Y,Past,0,0,0,...,Y,N,N,N,N,Y,N,0,65.0,1
2,93,Male,2.04,N,N,Y,Never,0,0,0,...,Y,N,N,N,N,Y,N,0,60.0,0
3,77,Male,2.30,N,Y,Y,Never,1,0,1,...,Y,N,N,N,N,Y,N,1,40.0,0
4,88,Male,2.30,N,Y,Y,Never,0,0,1,...,N,N,N,N,N,N,N,1,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,77,Male,2.30,N,N,N,Never,0,0,0,...,N,N,N,N,N,N,N,0,60.0,0
192,94,Female,1.79,N,Y,Y,Never,1,1,0,...,Y,N,N,Y,N,N,N,1,45.0,1
193,87,Female,1.66,N,Y,Y,Never,0,0,0,...,N,N,N,N,N,N,N,0,67.0,0
194,85,Male,2.20,N,Y,Y,Current,0,0,0,...,N,N,N,N,N,N,N,1,67.5,0


In [415]:
for col in x.columns:
    print(col)

Age (years)
Sex
BSA (m^2)
Diabetes
Hypertension
Hyperlipidemia
Smoking
Prior Myocardial Infarction
Prior PCI
Prior CABG
Prior Stroke or TIA
Peripheral Vascular Disease
Pulmonary Hypertentsion
Chronic obstructive Lung Disease
Home O2 Dependent
Renal insufficiency
Dialysis
Atrial fibrillation or flutter
Syncope
Obstructive coronary artery disease ≥ 2 vessels
Baseline left ventricle ejection fraction (%)
EndPointFinalized


In [416]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in x.columns:
    # check the datatype of the column, and if it is an object, encode it
    if x[column].dtype == object:
        label_encoders[column] = LabelEncoder()
        x[column] = label_encoders[column].fit_transform(x[column])

In [417]:
x.head()

,Age (years),Sex,BSA (m^2),Diabetes,Hypertension,Hyperlipidemia,Smoking,Prior Myocardial Infarction,Prior PCI,Prior CABG,...,Pulmonary Hypertentsion,Chronic obstructive Lung Disease,Home O2 Dependent,Renal insufficiency,Dialysis,Atrial fibrillation or flutter,Syncope,Obstructive coronary artery disease ≥ 2 vessels,Baseline left ventricle ejection fraction (%),EndPointFinalized
0,69,1,1.96,1,1,1,0,1,1,1,...,1,0,0,0,0,1,0,1,40.0,1
1,87,0,1.94,1,1,1,2,0,0,0,...,1,0,0,0,0,1,0,0,65.0,1
2,93,1,2.04,0,0,1,1,0,0,0,...,1,0,0,0,0,1,0,0,60.0,0
3,77,1,2.30,0,1,1,1,1,0,1,...,1,0,0,0,0,1,0,1,40.0,0
4,88,1,2.30,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,60.0,0


In [418]:
label_encoders.keys()

dict_keys(['Sex', 'Diabetes', 'Hypertension', 'Hyperlipidemia', 'Smoking', 'Prior Stroke or TIA', 'Peripheral Vascular Disease', 'Pulmonary Hypertentsion', 'Chronic obstructive Lung Disease', 'Home O2 Dependent', 'Renal insufficiency', 'Dialysis', 'Atrial fibrillation or flutter', 'Syncope'])

In [419]:
# verify no nans, and no object columns
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 22 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Age (years)                                      196 non-null    int64  
 1   Sex                                              196 non-null    int64  
 2   BSA (m^2)                                        196 non-null    float64
 3   Diabetes                                         196 non-null    int64  
 4   Hypertension                                     196 non-null    int64  
 5   Hyperlipidemia                                   196 non-null    int64  
 6   Smoking                                          196 non-null    int64  
 7   Prior Myocardial Infarction                      196 non-null    int64  
 8   Prior PCI                                        196 non-null    int64  
 9   Prior CABG                      

In [420]:
# Convert the data to numpy arrays for each sample/row.
x_data = x.to_numpy()
y_data = y.to_numpy()

In [421]:
print(x_data.shape, y_data.shape)

(196, 22) (196, 1)


In [422]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(156, 22) (156, 1) (40, 22) (40, 1)


In [423]:
y_val = y_val.reshape(-1)
y_train = y_train.reshape(-1)

In [426]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# model = GradientBoostingClassifier(
#     n_estimators=100, 
#     learning_rate=0.1, 
#     max_depth=3, 
#     random_state=42
# )
# model = RandomForestClassifier(
#     n_estimators=100, 
#     max_depth=3, 
#     random_state=42
# )
model = SVC(
    
)
model.fit(x_train, y_train)
# make predictions for test data
y_pred = model.predict(x_val)
predictions = np.round(y_pred)
# print(predictions, y_val)
# evaluate prediction
accuracy = accuracy_score(y_val, predictions)
recall = recall_score(y_val, predictions)
precision = precision_score(y_val, predictions)
f1 = f1_score(y_val, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))
print("Precision: %.2f%%" % (precision * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))

Accuracy: 100.00%
Recall: 100.00%
Precision: 100.00%
F1: 100.00%


In [427]:
from seaborn import heatmap
from sklearn.metrics import confusion_matrix, classification_report

In [428]:
report = classification_report(y_val, predictions)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       1.00      1.00      1.00        12

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40


In [429]:
print(confusion_matrix(y_val, predictions))

[[28  0]
 [ 0 12]]


In [430]:
import torch.nn as nn



In [431]:
import torch



In [432]:
x.head()

,Age (years),Sex,BSA (m^2),Diabetes,Hypertension,Hyperlipidemia,Smoking,Prior Myocardial Infarction,Prior PCI,Prior CABG,...,Pulmonary Hypertentsion,Chronic obstructive Lung Disease,Home O2 Dependent,Renal insufficiency,Dialysis,Atrial fibrillation or flutter,Syncope,Obstructive coronary artery disease ≥ 2 vessels,Baseline left ventricle ejection fraction (%),EndPointFinalized
0,69,1,1.96,1,1,1,0,1,1,1,...,1,0,0,0,0,1,0,1,40.0,1
1,87,0,1.94,1,1,1,2,0,0,0,...,1,0,0,0,0,1,0,0,65.0,1
2,93,1,2.04,0,0,1,1,0,0,0,...,1,0,0,0,0,1,0,0,60.0,0
3,77,1,2.30,0,1,1,1,1,0,1,...,1,0,0,0,0,1,0,1,40.0,0
4,88,1,2.30,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,60.0,0


In [433]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

network = nn.Sequential(
    nn.Linear(x_train.shape[1], 32),
    nn.SiLU(),
    # nn.Dropout(0.2),
    nn.Linear(32, 1),
)

loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.003, weight_decay=0.1)

epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = network(x_train_tensor).squeeze()
    loss_value = loss(output, y_train_tensor)
    loss_value.backward()
    optimizer.step()
    output = torch.round(torch.sigmoid(output)).detach().numpy()
    print(f"============ Train Epoch: {epoch} ============")
    print("Loss:", loss_value)
    print("Accuracy:", accuracy_score(y_train, output))
    print("Recall:", recall_score(y_train, output))
    print("Precision:", precision_score(y_train, output, zero_division=np.nan))
    print("F1:", f1_score(y_train, output))
    with torch.no_grad():
        output_val = network(x_val_tensor).squeeze()
        output_val = torch.round(torch.sigmoid(output_val))
        val_loss = loss(output_val, y_val_tensor)
        print(f"============ Val Epoch: {epoch} ============")
        print("Loss:", val_loss)
        print("Accuracy:", accuracy_score(y_val, output_val))
        print("Recall:", recall_score(y_val, output_val))
        print("Precision:", precision_score(y_val, output_val, zero_division=np.nan))
        print("F1:", f1_score(y_val, output_val))

============ Train Epoch: 0 ============
Loss: tensor(0.6885, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Accuracy: 0.717948717948718
Recall: 0.023255813953488372
Precision: 0.3333333333333333
F1: 0.043478260869565216
============ Val Epoch: 0 ============
Loss: tensor(0.7207)
Accuracy: 0.675
Recall: 0.16666666666666666
Precision: 0.4
F1: 0.23529411764705882
============ Train Epoch: 1 ============
Loss: tensor(0.6948, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Accuracy: 0.6666666666666666
Recall: 0.18604651162790697
Precision: 0.32
F1: 0.23529411764705882
============ Val Epoch: 1 ============
Loss: tensor(0.6931)
Accuracy: 0.7
Recall: 0.0
Precision: nan
F1: 0.0
============ Train Epoch: 2 ============
Loss: tensor(0.6617, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Accuracy: 0.717948717948718
Recall: 0.023255813953488372
Precision: 0.3333333333333333
F1: 0.043478260869565216
============ Val Epoch: 2 ============
Loss: tensor(0.6931)
Accuracy: 0.7
Recall: 0.0
Precisio

KeyboardInterrupt: 